# Instructions for calculating variables from pickup files

This notebook contains instructions for modifying the Fortran code to recalculate terms from the 3D pickup files. 

# 1) res_to_txt.f
We will use the file res_to_txt.f to calculate new terms. Most of the machinery is already in place.

Some points to note: 
 - CU1 is U velocity
 - CU2 is W velocity
 - CU3 is V velocity 

To convert to physical space call FFT_XY_TO_PHYSICAL

# 2) Execute: make -f ./Makefile_txt
From the LATMIX/Vapor/ folder.

# 3) Run ./res_to_txt



# Email from John regarding spectral coefficients

When the variables are in Fourier space, they should always be in the “packed Fourier” arrangement.  The only exception to this is inside the fft.f calls, but I don’t think you’re going to need to do anything in there.

The arrays are all equivalenced.  In other words, U1 and CU1 point to the same spots in memory.  That means that you shouldn’t use CU1 after calling FFT_XZ_TO_PHYSICAL(CU1,U1…)  and similar for U1 when the array is in Fourier space.  I found that doing this makes the FFTs faster and it obviously saves a factor of two in memory.  If you need both U1 and CU1 at the same time you could save CU1 to another array before doing the transform to physical space.

Regarding the number of non-zero Fourier modes in x and z, when transforming from physical space to Fourier space the first transform is in x.  This takes NX *real* numbers and transforms them into NX/2 *complex* numbers.  After de-aliasing we keep 2/3 of these, or NX/3.  Then, the next transform is a Fourier transform in z.  This now takes NZ *complex* numbers and transforms them into NZ *complex* numbers.  After de-aliasing, these are stored in TNKZ=2*NZ/3 non-zero wavenumbers.

>       do j=1,NY 
>          do i=0,NKX 
>            do k=0,TNKZ 
>            uw(i,k,j) = 
> CONJG(CU1(i,k,j))*dble(0.5d0*(CU2(i,k,j)+CU2(i,k,j+1))) 
>            end do 
>          end do 
>       end do 

You should be able to execute this right after the data is read in without needing to transform to physical space and back to Fourier space first.  The other comment is that I think you want:
> DBLE(   CONJG(CU1(i,k,j))  *  (0.5d0*(CU2(i,k,j)+CU2(i,k,j+1)))    )

You want to do the multiplication of the two complex numbers and then take the (double precision) real part with DBLE().  I’m assuming that uw is a double precision array.